# Setting up ELM physical models for calibration

In [1]:
import pickle

import dill
import rxmc

import elm

Using database version X4-2024-12-31 located in: /home/beyerk/db/exfor/unpack_exfor-2024/X4-2024-12-31


In [2]:
def elm_elastic(ws, *x):
    rxn = ws.reaction
    kinematics = ws.kinematics
    (
        central_isoscalar_params,
        central_isovector_params,
        spin_orbit_isoscalar_params,
        spin_orbit_isovector_params,
        coul_params,
        asym_factor,
    ) = elm.elm.calculate_parameters(
        tuple(rxn.projectile), tuple(rxn.target), kinematics.Ecm, rxn.Ef, *x
    )
    args_central = (
        asym_factor,
        central_isoscalar_params,
        central_isovector_params,
        coul_params,
    )
    args_spin_orbit = (
        asym_factor,
        spin_orbit_isoscalar_params,
        spin_orbit_isovector_params,
    )
    return args_central, args_spin_orbit

In [3]:
nn_model = rxmc.elastic_diffxs_model.ElasticDifferentialXSModel(
    "dXS/dA",
    elm.model_form.central_plus_coulomb, 
    elm.model_form.spin_orbit,
    elm_elastic,
    "ELM (n,n)",
)
pp_model = rxmc.elastic_diffxs_model.ElasticDifferentialXSModel(
    "dXS/dRuth",
    elm.model_form.central_plus_coulomb,
    elm.model_form.spin_orbit,
    elm_elastic,
    "ELM (p,p)",
)

In [4]:
with open("./elm_nn_model.pkl", "wb") as f:
    dill.dump(nn_model, f)
with open("./elm_pp_model.pkl", "wb") as f:
    dill.dump(pp_model, f)